<a href="https://colab.research.google.com/github/SWL75/Stable-Diffusion/blob/main/StableDiffusion_AUTOMATIC1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

WEB-UIの起動準備と拡張機能のインストール

In [ ]:
import os
import shutil
from google.colab import drive
import subprocess

drive.mount('/content/drive')

!pip uninstall --yes torch torchaudio torchvision torchtext
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2
!pip install xformers==0.0.25.post1 --index-url https://download.pytorch.org/whl/cu121

%cd /content/

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/stable-diffusion-webui
githubUrls = {
    'extensions': [
        'https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper',
        'https://github.com/AI-Creators-Society/stable-diffusion-webui-localization-ja_JP',
        'https://github.com/Mikubill/sd-webui-controlnet',
        'https://github.com/AlUlkesh/stable-diffusion-webui-images-browser',
        'https://github.com/Physton/sd-webui-prompt-all-in-one',
        'https://github.com/Bing-su/adetailer',


    ]
}

folder_path = '/content/stable-diffusion-webui/'

for category, urls in githubUrls.items():
    destination = folder_path+category
    for url in urls:

        repo_name = url.split('/')[-1]
        clone_path = os.path.join(destination, repo_name)
        if not os.path.exists(clone_path):
            subprocess.call(['git', 'clone', url, clone_path])
        else:
            print(f'{repo_name}はすでにダウンロードされています。 再度ダウンロードする必要がある場合は該当ファイルを削除してください。\n')
paths = {
    '/content/drive/MyDrive/StableDiffusion/CheckPoint': '/content/stable-diffusion-webui/models/Stable-diffusion',
    '/content/drive/MyDrive/StableDiffusion/Embeddings': '/content/stable-diffusion-webui/embeddings',
    '/content/drive/MyDrive/StableDiffusion/Lora': '/content/stable-diffusion-webui/models/Lora',
    '/content/drive/MyDrive/StableDiffusion/VAE': '/content/stable-diffusion-webui/models/VAE',
    '/content/drive/MyDrive/StableDiffusion/ControlNet': '/content/stable-diffusion-webui/models/ControlNet',
    '/content/drive/MyDrive/StableDiffusion/Config/config.json': '/content/stable-diffusion-webui/config.json',
    '/content/drive/MyDrive/StableDiffusion/Config/styles.csv':'/content/stable-diffusion-webui/styles.csv',
}

for path, link_path in paths.items():
    if not os.path.exists(path):
        print(f'リンク元のパスが存在しません: {path}')
        continue;
    if os.path.exists(link_path):

        if os.path.islink(link_path):
            continue
        elif os.path.isfile(link_path):
            os.remove(link_path)
        else:
            shutil.rmtree(link_path)


    !ln -s $path $link_path

!pip install insightface==0.7.3
!pip install gradio_client==0.2.7

!pip install fonts
!pip install font-roboto


WEB-UIの起動
↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。

In [ ]:
%cd /content/stable-diffusion-webui
!python launch.py --share --xformers --enable-insecure-extension-access

生成したデータ「Outputデータ」をGoogleドライブへコピー

In [ ]:
import os

output_folder = "/content/stable-diffusion-webui/output"
fallback_folder = "/content/stable-diffusion-webui/outputs"
zip_file = "/content/sd_output.zip"

if os.path.exists(output_folder):
    !zip -r {zip_file} {output_folder}
elif os.path.exists(fallback_folder):
    !zip -r {zip_file} {fallback_folder}
else:
    print(f"Neither '{output_folder}' nor '{fallback_folder}' found.")

if os.path.exists(zip_file):
    from google.colab import files
    files.download(zip_file)